# Pancake Problem: Baseline Submission

This notebook demonstrates a simple pancake-sort baseline that generates a Kaggle submission.
Moves use the prefix-reversal notation `Rk`, where `Rk` flips the first `k` elements of a state.

In [1]:
from __future__ import annotations

import pandas as pd
from pathlib import Path
from typing import Iterable

In [2]:
# Detect Kaggle environment ( /kaggle/input/... ), otherwise fall back to local repo layout.
DATA_ROOT = Path('/kaggle/input/pancake-sorting')
if not DATA_ROOT.exists():
    DATA_ROOT = Path('pancake-sorting')
assert DATA_ROOT.exists(), f'Dataset directory not found: {DATA_ROOT!s}'

TEST_PATH = DATA_ROOT / 'test.csv'
SUBMISSION_PATH = Path('submission.csv')  # Kaggle expects this in the working directory
TEST_PATH

PosixPath('/kaggle/input/pancake-sorting/test.csv')

In [3]:
def parse_permutation(raw: str) -> list[int]:
    """Parse a comma-separated permutation string into integer positions."""
    return [int(token) for token in raw.split(',') if token]

In [4]:
def pancake_sort_path(perm: Iterable[int]) -> list[str]:
    """Return a sequence of prefix reversals that sorts `perm` to the identity permutation."""
    arr = list(perm)
    n = len(arr)
    moves: list[str] = []

    for target in range(n, 1, -1):
        desired_value = target - 1
        idx = arr.index(desired_value)

        if idx == target - 1:
            continue  # already in place

        if idx != 0:
            moves.append(f'R{idx + 1}')
            arr[: idx + 1] = reversed(arr[: idx + 1])

        moves.append(f'R{target}')
        arr[:target] = reversed(arr[:target])

    return moves

In [5]:
test_df = pd.read_csv(TEST_PATH)
submission_rows: list[dict[str, str]] = []

for row in test_df.itertuples(index=False):
    perm = parse_permutation(row.permutation)
    moves = pancake_sort_path(perm)
    path_str = '.'.join(moves) if moves else 'UNSOLVED'
    submission_rows.append({'id': row.id, 'permutation': row.permutation, 'solution': path_str})

submission_df = pd.DataFrame(submission_rows).sort_values('id').reset_index(drop=True)
submission_df.head()

,id,permutation,solution
0,0,"3,2,0,1,4",R4.R2
1,1,"4,2,3,1,0",R5.R3.R4.R3.R2
2,2,"3,1,2,0,4",R4.R2.R3.R2
3,3,"1,2,0,3,4",R2.R3
4,4,"3,0,4,2,1",R3.R5.R3.R4


In [6]:
submission_df.to_csv(SUBMISSION_PATH, index=False)
SUBMISSION_PATH, len(submission_df)

(PosixPath('submission.csv'), 2405)

In [7]:
submission_df.sample(5, random_state=0)

,id,permutation,solution
2010,2010,"43,53,36,54,39,29,52,61,70,33,71,6,8,64,26,38,...",R27.R75.R30.R74.R47.R73.R42.R72.R29.R71.R15.R7...
2286,2286,"1,77,94,0,68,67,12,95,62,99,15,42,57,31,34,86,...",R10.R100.R81.R99.R52.R98.R10.R97.R50.R96.R91.R...
2100,2100,"30,45,2,15,28,47,61,59,17,4,60,51,5,16,73,38,3...",R52.R75.R38.R74.R54.R73.R40.R72.R16.R71.R52.R7...
2090,2090,"67,40,71,65,60,58,0,11,69,17,30,25,48,13,59,15...",R59.R75.R6.R74.R6.R73.R18.R72.R71.R23.R70.R3.R...
1070,1070,"7,16,25,29,6,18,11,3,24,13,0,2,9,1,26,4,10,21,...",R4.R30.R9.R29.R25.R28.R17.R27.R14.R26.R6.R25.R...
